# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS database1 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('database1')
except Exception as e:
    print(e)

## Task 1: Creating a table to model the query for getting the artist, song title and song's length for each session and item in session. To ensure all data are unique and returned from the table, sessionId and itemInSession are used as primary key.

In [13]:
## Query 1: Artist, song title and song's length in the music app history 
import pandas
df = pandas.read_csv("event_datafile_new.csv")
query = "CREATE TABLE IF NOT EXISTS session1 "
query = query + "( \
sessionId int, \
itemInSession int, \
artist text, \
length double, \
song text, \
PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "INSERT INTO session1 (sessionId, itemInSession, artist,  length, song)"
query = query + " VALUES (%s, %s, %s, %s, %s)"

for index, row in df.iterrows():
    try:
        session.execute(query, (row['sessionId'], row['itemInSession'], row['artist'], row['length'], row['song']))
    except Exception as e:
        print(e)

                    

In [14]:
## SELECT statement to verify the data was entered into the session1 
query = "select artist, song, length from session1 WHERE sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3073)


## Task 2: Creating a table to model query for retrieving  name of artist, song and user name in each itemInSession and sessionId. To ensure uniqueness, sessionId, userId and itemInSession as clustered columns are used as primary key.

In [19]:
## Query 2: Name of artist, song (sorted by itemInSession) and user (first and last name)
query = "CREATE TABLE IF NOT EXISTS user_songs "
query = query + "( \
sessionId int, \
userId int, \
itemInSession int, \
artist text, \
firstName text, \
lastName text, \
song text, \
PRIMARY KEY ((sessionId, userId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "INSERT INTO user_songs (sessionId, userId, itemInSession, artist, firstName, lastName, song)"
query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"

for index, row in df.iterrows():
    try:
        session.execute(query, (row['sessionId'], row['userId'], row['itemInSession'], row['artist'], row["firstName"], row["lastName"], row['song']))
    except Exception as e:
        print(e)
                    

In [20]:
## SELECT statement to verify the data was entered into user_songs
query = "select artist, song, firstName, lastName from user_songs WHERE userId=10 and sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row)    

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


##  Task 3: Creating a table to model query for retrieving users and the different songs they listened to. To ensure all records are retrieved, song an userID are used as primary key.

In [17]:
## Query 3: user name (first and last) in my music app history

query = "CREATE TABLE IF NOT EXISTS user_listener"
query = query + "( \
song text, \
userId int, \
firstName text, \
lastName text, \
PRIMARY KEY ((song), userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "INSERT INTO user_listener (song, userId, firstName, lastName)"
query = query + " VALUES (%s, %s, %s, %s)"

for index, row in df.iterrows():
    try:
        session.execute(query, (row['song'], row['userId'], row["firstName"], row["lastName"]))
    except Exception as e:
        print(e) 

In [18]:
## SELECT statement to verify the data was entered into user_listener 
query = "select firstName, lastName from user_listener WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [21]:
## TO-DO: Drop the table before closing out the sessions
try:
    session.execute("drop table session1")
except Exception as e:
    print(e)
    
try:
    session.execute("drop table user_songs")
except Exception as e:
    print(e)
    
try:
    session.execute("drop table user_listener")
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="unconfigured table session1"
Error from server: code=2200 [Invalid query] message="unconfigured table user_songs"


### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()